In [1]:
# created by Martti
#setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

gd_train=pd.read_csv("Data/Train_Plant1_Corrected.csv")
gd_test=pd.read_csv("Data/Test_Plant1_Corrected.csv")




In [2]:
gd_train1=pd.read_csv("Data/Train_Plant2.csv")
gd_test1=pd.read_csv("Data/Test_Plant2.csv")

In [3]:
gd_test

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,ARTIFICIAL
0,2020-06-10 00:00:00,1BY6WEcLGh8j5v7,22.842788,20.787240,0.0,4135001.0,0.0,0.0,932.000000,6439070.0,0
1,2020-06-10 00:00:00,1IF53ai7Xc0U56Y,22.842788,20.787240,0.0,4135001.0,0.0,0.0,0.000000,6380022.0,0
2,2020-06-10 00:00:00,3PZuoBAID5Wc2HD,22.842788,20.787240,0.0,4135001.0,0.0,0.0,0.000000,7183919.0,0
3,2020-06-10 00:00:00,7JYdWkrLSPkdwr4,22.842788,20.787240,0.0,4135001.0,0.0,0.0,1133.714286,7794729.0,0
4,2020-06-10 00:00:00,McdE0feGgRqW7Ca,22.842788,20.787240,0.0,4135001.0,0.0,0.0,0.000000,7355017.0,0
...,...,...,...,...,...,...,...,...,...,...,...
16891,2020-06-17 23:45:00,wCURE6d3bPkepu2,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5147.625000,7028601.0,0
16892,2020-06-17 23:45:00,z9Y9gH1T5YWrNuG,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5819.000000,7251204.0,0
16893,2020-06-17 23:45:00,zBIq5rxdHJRwDNY,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5817.000000,6583369.0,0
16894,2020-06-17 23:45:00,zVJPv84UY57bAof,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5910.000000,7363272.0,0


# NB! sõltuvalt Train/test switch

In [4]:
gd_train[48576:]
#24*4*22*3=6336
#53856-6336= 47520

#alates 2020-06-07

#Test andmestik
#gd=pd.concat([gd_train[48576:],gd_test], ignore_index=True)

#Train andmestik
gd=pd.concat([gd_train,gd_test[:1056]], ignore_index=True)

In [5]:
#Add columns
gd['DC_POWER_DIF_AVG_PER_ALL']=gd['DC_POWER']-gd['DC_POWER'].mean()
gd['AC_POWER_DIF_AVG_PER_ALL']=gd['AC_POWER']-gd['AC_POWER'].mean()
gd['DATE_TIME'] = pd.to_datetime(gd['DATE_TIME'], errors='coerce')
gd['DATE']=pd.to_datetime(gd['DATE_TIME'].dt.date)
gd['TIME']=gd['DATE_TIME'].dt.time

In [6]:
#New columns per inverter 
uurimiseks= gd.copy()
groups = dict(list(uurimiseks.groupby("SOURCE_KEY")))
shift_nr=3
shift_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']
gr_nr=0

#inverter based shifting
for group in groups:
    gr_nr+=1
    dates=groups[group]['DATE'].unique()
    for i in range(shift_nr):
        for j in range(len(shift_columns)):
            groups[group][shift_columns[j]+'_PER_INVERTER_MOMENT_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=(i+1), fill_value=0)
            groups[group][shift_columns[j]+'_PER_INVERTER_DAY_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=(i+1)*96, fill_value=0)
    
    #groups[group]['INVERTER']=group - ebaoluline
    groups[group]['DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    ############ per one day comparison, but note sure if correct
    gr_no2=0
    for date in dates:
        gr_no2 +=1
        d=groups[group][groups[group]['DATE']==date]
        d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
        d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])
        if gr_no2==1:
            date_data=pd.DataFrame(columns = d.keys())
            
        date_data =pd.concat([date_data, d], ignore_index=True)
    #########
    
    if gr_nr==1:
        gen_data=pd.DataFrame(columns = date_data.keys())
    
    #gen_data.append(groups[group], ignore_index=True) - did not seem to work and was replaced with concat
    gen_data=pd.concat([gen_data,date_data], ignore_index=True)
    
#gen_data=gen_data.fillna(value=0) - ebaoluline
gen_data

<ipython-input-6-e7170d9f046c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
<ipython-input-6-e7170d9f046c>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])


,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235
1,2020-05-15 00:15:00,1BY6WEcLGh8j5v7,25.084589,22.761668,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235
2,2020-05-15 00:30:00,1BY6WEcLGh8j5v7,24.935753,22.592306,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235
3,2020-05-15 00:45:00,1BY6WEcLGh8j5v7,24.846130,22.360852,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,25.184316,0.000000,22.857507,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235
4,2020-05-15 01:00:00,1BY6WEcLGh8j5v7,24.621525,22.165423,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,25.084589,0.000000,22.761668,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55963,2020-06-10 10:45:00,zVJPv84UY57bAof,27.744023,40.183040,0.428555,4135001.0,6032.142857,590.971429,1995.428571,7312451.429,...,26.920459,26.655896,36.647984,52.524404,0.319876,1.037922,2919.924576,286.676148,3579.180059,350.769903
55964,2020-06-10 11:00:00,zVJPv84UY57bAof,27.951821,38.822856,0.320387,4135001.0,4737.375000,464.800000,2126.375000,7312582.375,...,27.128528,27.075844,38.266939,53.649084,0.480099,0.848574,1625.156719,160.504719,2284.412202,224.598475
55965,2020-06-10 11:15:00,zVJPv84UY57bAof,27.866050,39.599638,0.601040,4135001.0,9770.428571,954.314286,2261.428571,7312717.429,...,27.596849,27.467251,42.387589,58.268178,0.510495,1.093312,6658.210290,650.019005,7317.465773,714.112760
55966,2020-06-10 11:30:00,zVJPv84UY57bAof,28.374143,47.206085,0.683142,4135001.0,8660.375000,846.587500,2510.000000,7312966.000,...,27.744023,27.859381,40.183040,57.171263,0.428555,0.855925,5548.156719,542.292219,6207.412202,606.385975


# Validation by comparison

In [7]:
#gen_data['AC_POWER'][20000:20004]

In [8]:
#gen_data['AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1'][20000:20004]

In [9]:
#a=gen_data[gen_data['SOURCE_KEY']=='1BY6WEcLGh8j5v7']
#b=pd.concat([a[a['DATE_TIME']=='2020-05-17 13:00:00'],a[a['DATE_TIME']=='2020-05-16 13:00:00']], ignore_index=True)
#b['AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1']

In [10]:
#b['AC_POWER']

In [11]:
#groups[group].keys()

# Other fields

In [12]:
#Group by day labels
'''
* label "FAULTY_according_to_DAILY_YIELD" - yield stays the same, while other inverters still increases
* label "INCORRECT_daily_yield" - yield per day decreases
* columns per inverter day average differences (total are done)
'''

uurimiseks= gen_data.copy()
groups = dict(list(uurimiseks.groupby("DATE")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_AVG_DIF_PER_DAY','AC_AVG_DIF_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_AVG_DIF_PER_DAY'] =  groups[group]['DC_POWER'] -np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_AVG_DIF_PER_DAY'] =  groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    gr_nr+=1
    '''
    Tundub et toodaks vale infot
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_DAY_SHIFT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data2=pd.DataFrame(columns =groups[group].keys())
    
    gen_data2=pd.concat([gen_data2,groups[group]], ignore_index=True)
    
gen_data2

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,0.000000,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235,-2664.412473,-260.810113
1,2020-05-15 00:15:00,1BY6WEcLGh8j5v7,25.084589,22.761668,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,0.000000,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235,-2664.412473,-260.810113
2,2020-05-15 00:30:00,1BY6WEcLGh8j5v7,24.935753,22.592306,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,0.000000,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235,-2664.412473,-260.810113
3,2020-05-15 00:45:00,1BY6WEcLGh8j5v7,24.846130,22.360852,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,22.857507,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235,-2664.412473,-260.810113
4,2020-05-15 01:00:00,1BY6WEcLGh8j5v7,24.621525,22.165423,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,22.761668,0.000000,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235,-2664.412473,-260.810113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55963,2020-06-10 10:45:00,zVJPv84UY57bAof,27.744023,40.183040,0.428555,4135001.0,6032.142857,590.971429,1995.428571,7312451.429,...,36.647984,52.524404,0.319876,1.037922,2919.924576,286.676148,3579.180059,350.769903,3620.866359,354.827548
55964,2020-06-10 11:00:00,zVJPv84UY57bAof,27.951821,38.822856,0.320387,4135001.0,4737.375000,464.800000,2126.375000,7312582.375,...,38.266939,53.649084,0.480099,0.848574,1625.156719,160.504719,2284.412202,224.598475,2326.098502,228.656120
55965,2020-06-10 11:15:00,zVJPv84UY57bAof,27.866050,39.599638,0.601040,4135001.0,9770.428571,954.314286,2261.428571,7312717.429,...,42.387589,58.268178,0.510495,1.093312,6658.210290,650.019005,7317.465773,714.112760,7359.152073,718.170405
55966,2020-06-10 11:30:00,zVJPv84UY57bAof,28.374143,47.206085,0.683142,4135001.0,8660.375000,846.587500,2510.000000,7312966.000,...,40.183040,57.171263,0.428555,0.855925,5548.156719,542.292219,6207.412202,606.385975,6249.098502,610.443620


In [13]:
#groups[group].keys()

In [14]:
# add inverter difference from average per that moment
'''
* columns "MOMENT_AVERAGE" - average value per this moment of inverter production info
* columns "DIF_FROM_AVERAGE" - inverter avg values in comparison with other inverters per this moment
'''

uurimiseks= gen_data2.copy()
groups = dict(list(uurimiseks.groupby("DATE_TIME")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_DIF_AVG_PER_DAY','AC_DIF_AVG_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_DIF_AVG_PER_MOMENT'] =  groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_MOMENT'] =  groups[group]['AC_POWER'] -np.mean(groups[group]['AC_POWER']) 

    gr_nr+=1
    ''' 
    tundub valet tulemust andvat
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_MOMENT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data3=pd.DataFrame(columns =groups[group].keys())
    
    gen_data3=pd.concat([gen_data3,groups[group]], ignore_index=True)
    
gen_data3

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY,DC_DIF_AVG_PER_MOMENT,AC_DIF_AVG_PER_MOMENT
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.000000,4135001.0,0.000000,0.000000,0.000000,6259559.000,...,0.000000,0.000000,-2863.146291,-280.049126,-2451.465588,-240.068235,-2664.412473,-260.810113,0.000000,0.000000
1,2020-05-15 00:00:00,1IF53ai7Xc0U56Y,25.184316,22.857507,0.000000,4135001.0,0.000000,0.000000,0.000000,6183645.000,...,0.000000,0.000000,-3137.970674,-306.803297,-2696.990699,-263.992560,-2664.412473,-260.810113,0.000000,0.000000
2,2020-05-15 00:00:00,3PZuoBAID5Wc2HD,25.184316,22.857507,0.000000,4135001.0,0.000000,0.000000,0.000000,6987759.000,...,0.000000,0.000000,-3138.156236,-306.824023,-2653.813058,-259.767690,-2664.412473,-260.810113,0.000000,0.000000
3,2020-05-15 00:00:00,7JYdWkrLSPkdwr4,25.184316,22.857507,0.000000,4135001.0,0.000000,0.000000,0.000000,7602960.000,...,0.000000,0.000000,-3062.983466,-299.508297,-2610.503534,-255.561570,-2664.412473,-260.810113,0.000000,0.000000
4,2020-05-15 00:00:00,McdE0feGgRqW7Ca,25.184316,22.857507,0.000000,4135001.0,0.000000,0.000000,0.000000,7158964.000,...,0.000000,0.000000,-3125.895334,-305.659463,-2750.322731,-269.173270,-2664.412473,-260.810113,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55963,2020-06-10 11:45:00,uHbuxQJl8lW7ozc,28.499033,45.567177,0.685318,4135001.0,10616.625000,1036.562500,2778.875000,7236641.875,...,0.320387,0.826397,7497.391317,731.572752,8118.260045,791.965476,8205.348502,800.418620,1474.633928,143.409903
55964,2020-06-10 11:45:00,wCURE6d3bPkepu2,28.499033,45.567177,0.685318,4135001.0,7578.375000,741.475000,2675.250000,6978655.250,...,0.320387,0.826397,4488.083524,439.315587,5210.495164,509.553013,5167.098502,505.331120,-1563.616072,-151.677597
55965,2020-06-10 11:45:00,z9Y9gH1T5YWrNuG,28.499033,45.567177,0.685318,4135001.0,7296.875000,713.600000,2679.000000,7201732.000,...,0.320387,0.826397,4241.745089,414.866568,4936.008929,482.362165,4885.598502,477.456120,-1845.116072,-179.552597
55966,2020-06-10 11:45:00,zBIq5rxdHJRwDNY,28.499033,45.567177,0.685318,4135001.0,10293.428570,1005.114286,2694.285714,6534099.286,...,0.320387,0.826397,7228.166574,705.373641,7890.187499,769.778200,7882.152072,768.970406,1151.437498,111.961689


#other labels adding
'''
* label "INCORRECT_total_yield" - yield per period decreases
* label "INCORRECT_according_to_DC_vs_AC" - DC and AC ratio is off
* label "INCORRECT_according_to_DC_AC_DAILY_YIELD" - DC and AC increases, while DAILY_YIELD does not
'''

In [15]:
#Test andmestik (drop first three train days)
#gen_data3=gen_data3[6336:]

#Train andmestik - nill

In [16]:
gen_data3.to_csv(path_or_buf='Data/Train_Plant1_added_features.csv')

In [17]:
#katse=pd.read_csv("Data/Test_Plant1_added_features.csv")

In [18]:
#katse